# TripletCough: Cougher Identification and Verification from Contact-Free Smartphone-Based Audio Recordings Using Metric Learning
We have already trained our TripletCough network on a data set of voluntary coughs recorded using the RØDE NT1000 studio microphone, as described in the paper. We provide you with the pre-trained model file in `weights_testing/rode_close/20201108_065654__Parameters_rode_close_weights.h5`. The code used to evaluate this pre-trained model can be found in the  section "Python Scripts" which explains how to run the various python scripts available for performing the identification and verification tests that are reported in the paper.

# Verification Task

In [3]:
# Libraries
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

# Define the relevant variables (number of enrollment and test samples, etc.)
nb_participants = 20
nb_enrollment_samples = 10
nb_test_samples = 5

# Define the expected size of the spectrogram
spect_height = 80
spect_width = 237

# Define threshold for decision rule
THRESHOLD = 2.2

# Load Model
from Model import get_triplet_network, get_embedding_cnn
model = get_triplet_network((spect_height, spect_width, 1))
embedding_cnn = get_embedding_cnn((spect_height, spect_width, 1))

# 20 participants with 10 enrollment samples each
X_enrollment = np.random.rand(nb_participants, nb_enrollment_samples, spect_height, spect_width)
n_samples_enrollment = [nb_enrollment_samples] * nb_participants

# 20 participants with 5 test samples each
X_test = np.random.rand(nb_participants, nb_test_samples, spect_height, spect_width)
n_samples_test = [nb_test_samples] * nb_participants

weights_file = os.path.join(".", "weights_testing", "rode_close", "20201108_065654__Parameters_rode_close_weights.h5")

# Load weights
model.load_weights(weights_file)
embedding_cnn.set_weights(model.get_weights())

tp, fp, tn, fn = 4 * [0]

for participant in range(nb_participants):
    # Select the remaining n_test samples (different from the enrollment samples) from P1 as test samples
    remaining_participant_list = list(range(nb_participants))
    remaining_participant_list.remove(participant)

    # Retrieve images from P1
    P1_enrollment = X_enrollment[participant, :, :, :].reshape(
    nb_enrollment_samples, spect_height, spect_width, 1
    )
    P1_test = X_test[participant, :, :, :].reshape(nb_test_samples, spect_height, spect_width, 1)

    test_samples = X_test[remaining_participant_list, :, :, :].reshape(len(remaining_participant_list * nb_test_samples), spect_height, spect_width, 1)

    # Compute the embeddings
    emb_P1_enrollment = embedding_cnn.predict(P1_enrollment)
    emb_P1_test = embedding_cnn.predict(P1_test)
    emb_others = embedding_cnn.predict(test_samples)

    m = tf.keras.metrics.MeanTensor()
    
    for test_sample in emb_P1_test:
        for enrollment_sample in emb_P1_enrollment:
            distance = K.sum(K.square(test_sample - enrollment_sample), axis=0)
            m.update_state(distance)
        mean_distance = m.result()
        if mean_distance < THRESHOLD:
            tp += 1
        else:
            fn += 1
        m.reset_states()

    for test_sample in emb_others:
        for enrollment_sample in emb_P1_enrollment:
            distance = K.sum(K.square(test_sample - enrollment_sample), axis=0)
            m.update_state(distance)
        mean_distance = m.result()
        if mean_distance < THRESHOLD:
            fp += 1
        else:
            tn += 1
        m.reset_states()
    
acc = (tp + tn) / (tp + tn + fp + fn)
far = fp / (fp + tn)
frr = fn / (fn + tp)

print("ACC: ", acc)
print("FAR: ", far)
print("FRR: ", frr)


ACC:  0.05
FAR:  1.0
FRR:  0.0


# Identification Task (2-way 4-shot)

In [5]:
from Model import get_triplet_network, get_embedding_cnn
from itertools import permutations
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy.random as rng

nb_participants = 20
nb_enrollment_samples = 10
nb_test_samples = 5
spect_height = 80
spect_width = 237

# Identification task
n_tasks = 5
k = 4 # 4-shot evaluation

# Load Model
model = get_triplet_network((spect_height, spect_width, 1))
embedding_cnn = get_embedding_cnn((spect_height, spect_width, 1))

# Generate the Testing Data from Pickle File
# 20 participants with 10 enrollment samples each
X = np.random.rand(nb_participants, nb_enrollment_samples, spect_height, spect_width)
n_samples = [nb_enrollment_samples] * nb_participants

# Write header line to csv file
permutation_list = list(permutations(range(X.shape[0]), 2))
permutation_list.insert(0, "Participant Combinations")

# Loaded the pre-trained weight file
weights_path = os.path.join(".", "weights_testing", "rode_close", "20201108_065654__Parameters_rode_close_weights.h5")
np.random.seed(0)

# Load weights
model.load_weights(weights_path)
embedding_cnn.set_weights(model.get_weights())

acc_model = []
count = 0
# Loop over all participant combinations
for participant_combo in permutations(range(X.shape[0]), 2):
    n_test = 0
    n_correct = 0

    # For each pair of participants, generate n_tasks (default: 100) random 2-way-k-shot tasks and
    # evaluate accuracy over these tasks
    for i in range(0, n_tasks):

        # First select k+1 samples u.a.r. from P1
        P1_samples = rng.choice(range(n_samples[participant_combo[0]]), size=(k + 1,), replace=False)
        # Select first sample from P1 as test sample
        P1_test_sample = P1_samples[0]
        # Select the other k samples (different from the test sample) from P1 as anchor samples
        P1_anchor_samples = P1_samples[1:]

        # Select k samples u.a.r. from P2 as anchor samples
        P2_anchor_samples = rng.choice(range(n_samples[participant_combo[1]]), size=(k,), replace=False)

        # Retrieve images from test and anchor samples
        test_img = X[participant_combo[0], P1_test_sample, :, :].reshape(1, spect_height, spect_width, 1)

        P1_anchor_imgs = X[participant_combo[0], P1_anchor_samples, :, :].reshape(k, spect_height, spect_width, 1)
        P2_anchor_imgs = X[participant_combo[1], P2_anchor_samples, :, :].reshape(k, spect_height, spect_width, 1)

        # Create support set composed of k P1 and k P2 anchor images
        support_set = np.concatenate((P1_anchor_imgs, P2_anchor_imgs), axis=0)

        # Test model
        # Compute embeddings for test sample and for each anchor sample in support set
        embedding_test_img = embedding_cnn.predict(test_img)
        embedding_support_set = embedding_cnn.predict(support_set)

        distances = []

        # Compute distances between embeddings of test sample and each anchor sample in support set
        for emb in embedding_support_set:
            distances.append(K.sum(K.square(embedding_test_img - emb), axis=1))

        # Compute mean distance between test sample and anchor samples of P1 / test sample and anchor samples of P2
        m = tf.keras.metrics.MeanTensor()

        for i in range(0, k):
            m.update_state(distances[i])

        P1_mean_distance = m.result()

        m.reset_states()

        for i in range(k, len(distances)):
            m.update_state(distances[i])

        P2_mean_distance = m.result()

        if P1_mean_distance < P2_mean_distance:
            n_correct += 1

    # Compute k-shot test accuracy for this participant combination
    acc = 100.0 * n_correct / n_tasks
    print("ACC: ", acc)
    acc_model.append(acc)
    count += n_tasks

print()
print("Mean ACC: ", (sum(acc_model)/len(acc_model)))


ACC:  40.0
ACC:  20.0
ACC:  0.0
ACC:  20.0
ACC:  60.0
ACC:  40.0
ACC:  60.0
ACC:  80.0
ACC:  60.0
ACC:  60.0
ACC:  60.0
ACC:  20.0
ACC:  100.0
ACC:  20.0
ACC:  40.0
ACC:  60.0
ACC:  20.0
ACC:  60.0
ACC:  20.0
ACC:  60.0
ACC:  40.0
ACC:  80.0
ACC:  0.0
ACC:  100.0
ACC:  40.0
ACC:  40.0
ACC:  40.0
ACC:  60.0
ACC:  40.0
ACC:  80.0
ACC:  40.0
ACC:  100.0
ACC:  40.0
ACC:  60.0
ACC:  40.0
ACC:  60.0
ACC:  40.0
ACC:  60.0
ACC:  60.0
ACC:  40.0
ACC:  40.0
ACC:  40.0
ACC:  60.0
ACC:  20.0
ACC:  100.0
ACC:  40.0
ACC:  80.0
ACC:  60.0
ACC:  40.0
ACC:  20.0
ACC:  60.0
ACC:  20.0
ACC:  40.0
ACC:  20.0
ACC:  100.0
ACC:  20.0
ACC:  40.0
ACC:  80.0
ACC:  40.0
ACC:  60.0
ACC:  40.0
ACC:  80.0
ACC:  100.0
ACC:  60.0
ACC:  80.0
ACC:  80.0
ACC:  60.0
ACC:  40.0
ACC:  60.0
ACC:  100.0
ACC:  60.0
ACC:  80.0
ACC:  20.0
ACC:  60.0
ACC:  40.0
ACC:  40.0
ACC:  60.0
ACC:  60.0
ACC:  80.0
ACC:  60.0
ACC:  40.0
ACC:  80.0
ACC:  80.0
ACC:  100.0
ACC:  40.0
ACC:  80.0
ACC:  80.0
ACC:  100.0
ACC:  100.0
ACC:  60.0
AC